<h2 align='center'>IPL MATCH WIN PREDICTOR</h2>

In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  
sns.set()

import warnings
warnings.filterwarnings('ignore')

In [14]:
matches = pd.read_csv('match_info_data.csv')
deliveries = pd.read_csv('match_data.csv')
matches.head(2)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1370353,2023,Ahmedabad,2023/05/29,Gujarat Titans,Chennai Super Kings,Chennai Super Kings,field,D/L,1,Chennai Super Kings,0,5,DP Conway,"Narendra Modi Stadium, Ahmedabad",Nitin Menon,RJ Tucker,KN Ananthapadmanabhan
1,1370352,2023,Ahmedabad,2023/05/26,Gujarat Titans,Mumbai Indians,Mumbai Indians,field,normal,0,Gujarat Titans,62,0,Shubman Gill,"Narendra Modi Stadium, Ahmedabad",Nitin Menon,RJ Tucker,J Madanagopal


In [13]:
deliveries.head(2)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id
0,1370353,2023,2023-05-29,"Narendra Modi Stadium, Ahmedabad",1,0.1,Gujarat Titans,Chennai Super Kings,WP Saha,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370353
1,1370353,2023,2023-05-29,"Narendra Modi Stadium, Ahmedabad",1,0.2,Gujarat Titans,Chennai Super Kings,WP Saha,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370353


In [11]:
matches.shape, deliveries.shape

((1024, 18), (243817, 23))

In [15]:
matches.columns

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [16]:
matches['season'].value_counts()

season
2013       76
2023       74
2022       74
2012       74
2011       73
2021       60
2020/21    60
2019       60
2018       60
2016       60
2014       60
2009/10    60
2017       59
2015       59
2007/08    58
2009       57
Name: count, dtype: int64

In [19]:
matches['city'].nunique()

35

In [20]:
deliveries.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'cricsheet_id'],
      dtype='object')

In [25]:
deliveries['match_id'].nunique()

1024

In [28]:
totalrun_df = deliveries.groupby(['match_id', 'innings'])['runs_off_bat'].sum().reset_index()
totalrun_df

,match_id,innings,runs_off_bat
0,335982,1,205
1,335982,2,63
2,335983,1,234
3,335983,2,196
4,335984,1,122
...,...,...,...
2070,1370351,2,91
2071,1370352,1,223
2072,1370352,2,164
2073,1370353,1,210


In [29]:
totalrun_df['innings'].nunique()

6

In [30]:
totalrun_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   match_id      2075 non-null   int64
 1   innings       2075 non-null   int64
 2   runs_off_bat  2075 non-null   int64
dtypes: int64(3)
memory usage: 48.8 KB


In [34]:
# capturing only the first innings, as we well be  predicting for the second innings

# Filtering rows where inning equals 1
totalrun_df = totalrun_df[totalrun_df['innings'] == 1]

# Incrementing the values in the total_runs column by 1
totalrun_df['runs_off_bat'] = totalrun_df['runs_off_bat'].apply(lambda x: x + 1)
totalrun_df

,match_id,innings,runs_off_bat
0,335982,1,206
2,335983,1,235
4,335984,1,123
6,335985,1,155
8,335986,1,101
...,...,...,...
2065,1359544,1,191
2067,1370350,1,168
2069,1370351,1,167
2071,1370352,1,224


In [35]:
matches.columns

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [40]:
'''
mergin the total first innings score df with the matches df,
where left side merging is done on 'id columns for the matches and
righe side merging is done on 'match_id' column of the totalrun_df 
'''

match_df = matches.merge(totalrun_df[['match_id', 'runs_off_bat']], left_on='id', right_on='match_id')
match_df.head(2)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,runs_off_bat
0,1370353,2023,Ahmedabad,2023/05/29,Gujarat Titans,Chennai Super Kings,Chennai Super Kings,field,D/L,1,Chennai Super Kings,0,5,DP Conway,"Narendra Modi Stadium, Ahmedabad",Nitin Menon,RJ Tucker,KN Ananthapadmanabhan,1370353,211
1,1370352,2023,Ahmedabad,2023/05/26,Gujarat Titans,Mumbai Indians,Mumbai Indians,field,normal,0,Gujarat Titans,62,0,Shubman Gill,"Narendra Modi Stadium, Ahmedabad",Nitin Menon,RJ Tucker,J Madanagopal,1370352,224


In [42]:
match_df['team1'].unique()

array(['Gujarat Titans', 'Mumbai Indians', 'Chennai Super Kings',
       'Sunrisers Hyderabad', 'Royal Challengers Bangalore',
       'Lucknow Super Giants', 'Punjab Kings', 'Delhi Capitals',
       'Kolkata Knight Riders', 'Rajasthan Royals', 'Kings XI Punjab',
       'Delhi Daredevils', 'Rising Pune Supergiant', 'Gujarat Lions',
       'Rising Pune Supergiants', 'Pune Warriors', 'Deccan Chargers',
       'Kochi Tuskers Kerala'], dtype=object)

In [43]:
match_df['team2'].unique()

array(['Chennai Super Kings', 'Mumbai Indians', 'Lucknow Super Giants',
       'Gujarat Titans', 'Kolkata Knight Riders', 'Delhi Capitals',
       'Rajasthan Royals', 'Royal Challengers Bangalore', 'Punjab Kings',
       'Sunrisers Hyderabad', 'Kings XI Punjab', 'Delhi Daredevils',
       'Rising Pune Supergiant', 'Gujarat Lions',
       'Rising Pune Supergiants', 'Pune Warriors', 'Deccan Chargers',
       'Kochi Tuskers Kerala'], dtype=object)

In [44]:
teams = ['Chennai Super Kings', 'Mumbai Indians', 'Lucknow Super Giants',
       'Gujarat Titans', 'Kolkata Knight Riders', 'Delhi Capitals',
       'Rajasthan Royals', 'Royal Challengers Bangalore', 'Punjab Kings',
       'Sunrisers Hyderabad'
       ]

In [46]:
# replacing the 'Delhi Daredevils' with 'Delhi Capitals'
match_df['team1']= match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2']= match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

#replacing the Kings XI Punjab' with 'Punjab Kings'
match_df['team1']= match_df['team1'].str.replace('Kings XI Punjab','Punjab Kings')
match_df['team2']= match_df['team2'].str.replace('Kings XI Punjab','Punjab Kings')

#replacing the 'Deccan Chargers' with 'Sunrisers Hyderabad'
match_df['team1']= match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2']= match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [49]:
match_df['team1'].unique()

array(['Gujarat Titans', 'Mumbai Indians', 'Chennai Super Kings',
       'Sunrisers Hyderabad', 'Royal Challengers Bangalore',
       'Lucknow Super Giants', 'Punjab Kings', 'Delhi Capitals',
       'Kolkata Knight Riders', 'Rajasthan Royals',
       'Rising Pune Supergiant', 'Gujarat Lions',
       'Rising Pune Supergiants', 'Pune Warriors', 'Kochi Tuskers Kerala'],
      dtype=object)

In [50]:
match_df['team2'].unique()

array(['Chennai Super Kings', 'Mumbai Indians', 'Lucknow Super Giants',
       'Gujarat Titans', 'Kolkata Knight Riders', 'Delhi Capitals',
       'Rajasthan Royals', 'Royal Challengers Bangalore', 'Punjab Kings',
       'Sunrisers Hyderabad', 'Rising Pune Supergiant', 'Gujarat Lions',
       'Rising Pune Supergiants', 'Pune Warriors', 'Kochi Tuskers Kerala'],
      dtype=object)

In [51]:
match_df.shape

(1024, 20)

In [54]:
teams

['Chennai Super Kings',
 'Mumbai Indians',
 'Lucknow Super Giants',
 'Gujarat Titans',
 'Kolkata Knight Riders',
 'Delhi Capitals',
 'Rajasthan Royals',
 'Royal Challengers Bangalore',
 'Punjab Kings',
 'Sunrisers Hyderabad']

In [52]:
# will consider only frequently occuring teams, which are mentioned in the team list

match_df = match_df[match_df['team1'].isin(teams)]
match_df =match_df[match_df['team2'].isin(teams)]

In [53]:
match_df.shape

(909, 20)

In [55]:
match_df['team1'].value_counts()

team1
Royal Challengers Bangalore    127
Chennai Super Kings            116
Punjab Kings                   112
Delhi Capitals                 111
Mumbai Indians                 108
Sunrisers Hyderabad            108
Kolkata Knight Riders          105
Rajasthan Royals                92
Gujarat Titans                  15
Lucknow Super Giants            15
Name: count, dtype: int64

In [56]:
match_df['team2'].value_counts()

team2
Mumbai Indians                 122
Kolkata Knight Riders          117
Sunrisers Hyderabad            117
Delhi Capitals                 111
Rajasthan Royals               107
Punjab Kings                   105
Chennai Super Kings            100
Royal Challengers Bangalore     97
Gujarat Titans                  18
Lucknow Super Giants            15
Name: count, dtype: int64